In [1]:
from glob import glob
import pandas as pd

In [2]:
files_dict = {filename.split('/')[-1].split('.')[0]: filename for filename in glob('results/orf_x_ref/*')}
dfs = {key: pd.read_csv(file_path) for key, file_path in files_dict.items()}

In [3]:
dfs['rdrp_orf1_orf_x_ref'].sort_values(by='identity_score').head(20)

,SRR,contig_name,orf_name,ref_name,identity_score,gap_openings,N/orf_len,M,N,aln_len,orf_len,ref_len
456,SRR4435988,NODE_216_length_6091_cov_154.402727_SRR4435988,NODE_216_length_6091_cov_154.402727_SRR4435988_4,NC_003610.1_rdrp_orf1_[Kyuri_green_mottle_mosa...,0.248864,76,0.820896,661,880,1358,1072,1166
610,SRR5087335,NODE_1031_length_5306_cov_347.287374_SRR5087335,NODE_1031_length_5306_cov_347.287374_SRR5087335_6,MW346736.1_rdrp_orf1_[Virgaviridae_sp.],0.250282,73,0.780810,665,887,1366,1136,1117
585,SRR5087335,NODE_1031_length_5306_cov_347.287374_SRR5087335,NODE_1031_length_5306_cov_347.287374_SRR5087335_6,NC_003630.1_rdrp_orf1_[Pepper_mild_mottle_virus],0.250282,73,0.780810,665,887,1366,1136,1117
385,SRR4435986,NODE_278_length_4826_cov_1224.271215_SRR4435986,NODE_278_length_4826_cov_1224.271215_SRR4435986_3,NC_003610.1_rdrp_orf1_[Kyuri_green_mottle_mosa...,0.250844,76,0.822387,666,889,1358,1081,1166
428,SRR4435988,NODE_216_length_6091_cov_154.402727_SRR4435988,NODE_216_length_6091_cov_154.402727_SRR4435988_4,NC_040685.2_rdrp_orf1_[Opuntia_virus_2],0.252273,71,0.820896,658,880,1292,1072,1100
357,SRR4435986,NODE_278_length_4826_cov_1224.271215_SRR4435986,NODE_278_length_4826_cov_1224.271215_SRR4435986_3,NC_040685.2_rdrp_orf1_[Opuntia_virus_2],0.253121,71,0.814986,658,881,1300,1081,1100
624,SRR5087335,NODE_1031_length_5306_cov_347.287374_SRR5087335,NODE_1031_length_5306_cov_347.287374_SRR5087335_6,AJ132576.1_rdrp_orf1_[Soil-borne_cereal_mosaic...,0.253918,71,0.842430,714,957,1487,1136,1308
499,SRR4435991,NODE_75_length_6147_cov_733.997199_SRR4435991,NODE_75_length_6147_cov_733.997199_SRR4435991_1,NC_040685.2_rdrp_orf1_[Opuntia_virus_2],0.254257,71,0.808999,657,881,1308,1089,1100
536,SRR4435991,NODE_75_length_6147_cov_733.997199_SRR4435991,NODE_75_length_6147_cov_733.997199_SRR4435991_1,OL404963.1_rdrp_orf1_[Trichosanthes_mottle_mos...,0.256264,69,0.806244,653,878,1377,1089,1166
465,SRR4435988,NODE_216_length_6091_cov_154.402727_SRR4435988,NODE_216_length_6091_cov_154.402727_SRR4435988_4,OL404963.1_rdrp_orf1_[Trichosanthes_mottle_mos...,0.256381,68,0.804104,641,862,1376,1072,1166


In [4]:
dfs['rdrp_orf2_orf_x_ref']

,SRR,contig_name,orf_name,ref_name,identity_score,gap_openings,N/orf_len,M,N,aln_len,orf_len,ref_len
0,DRR146894,NODE_4230_length_1601_cov_1210.913325_DRR146894,NODE_4230_length_1601_cov_1210.913325_DRR146894_1,NC_001728.1_rdrp_orf2_[Odontoglossum_ringspot_...,0.387387,13,0.892761,204,333,498,373,458
1,DRR146894,NODE_4230_length_1601_cov_1210.913325_DRR146894,NODE_4230_length_1601_cov_1210.913325_DRR146894_1,NC_009642.1_rdrp_orf2_[Bell_pepper_mottle_virus],0.382090,11,0.898123,207,335,502,373,464
2,DRR146894,NODE_4230_length_1601_cov_1210.913325_DRR146894,NODE_4230_length_1601_cov_1210.913325_DRR146894_1,NC_040685.2_rdrp_orf2_[Opuntia_virus_2],0.379538,9,0.812332,188,303,382,373,312
3,DRR146894,NODE_4230_length_1601_cov_1210.913325_DRR146894,NODE_4230_length_1601_cov_1210.913325_DRR146894_1,NC_016519.1_rdrp_orf2_[Clitoria_yellow_mottle_...,0.348974,12,0.914209,222,341,467,373,435
4,DRR146894,NODE_4230_length_1601_cov_1210.913325_DRR146894,NODE_4230_length_1601_cov_1210.913325_DRR146894_1,NC_001556.1_rdrp_orf2_[Tobacco_mild_green_mosa...,0.355623,10,0.882038,212,329,538,373,494
...,...,...,...,...,...,...,...,...,...,...,...,...
20727,SRR9960038,NODE_10536_length_2129_cov_1.942008_SRR9960038,NODE_10536_length_2129_cov_1.942008_SRR9960038_2,AJ238607.1_rdrp_orf2_[Potato_mop-top_virus],0.264789,29,0.855422,261,355,559,415,499
20728,SRR9960038,NODE_10536_length_2129_cov_1.942008_SRR9960038,NODE_10536_length_2129_cov_1.942008_SRR9960038_2,KT225277.1_rdrp_orf2_[Soil-borne_virus_2],0.271429,30,0.843373,255,350,543,415,478
20729,SRR9960038,NODE_10536_length_2129_cov_1.942008_SRR9960038,NODE_10536_length_2129_cov_1.942008_SRR9960038_2,X14006.1_rdrp_orf2_[Pea_early_browning_virus],0.270186,22,0.775904,235,322,564,415,471
20730,SRR9960038,NODE_10536_length_2129_cov_1.942008_SRR9960038,NODE_10536_length_2129_cov_1.942008_SRR9960038_2,L23972.1_rdrp_orf2_[Pepper_ringspot_virus],0.286164,26,0.766265,227,318,567,415,470


In [5]:
dfs['cp_orf_x_ref']

,SRR,contig_name,orf_name,ref_name,identity_score,gap_openings,N/orf_len,M,N,aln_len,orf_len,ref_len
0,DRR146894,NODE_4230_length_1601_cov_1210.913325_DRR146894,NODE_4230_length_1601_cov_1210.913325_DRR146894_2,NP_056812.1_18KDa_coat_protein_[Odontoglossum_...,0.205674,7,0.886792,112,141,176,159,158
1,DRR146894,NODE_4230_length_1601_cov_1210.913325_DRR146894,NODE_4230_length_1601_cov_1210.913325_DRR146894_2,YP_001333653.1_coat_protein_[Bell_pepper_mottl...,0.194444,5,0.905660,116,144,175,159,160
2,DRR146894,NODE_4230_length_1601_cov_1210.913325_DRR146894,NODE_4230_length_1601_cov_1210.913325_DRR146894_2,YP_009553016.2_coat_protein_[Opuntia_virus_2],0.251799,9,0.874214,104,139,199,159,179
3,DRR146894,NODE_4230_length_1601_cov_1210.913325_DRR146894,NODE_4230_length_1601_cov_1210.913325_DRR146894_2,YP_004956730.1_unnamed_protein_product_[Clitor...,0.260870,8,0.867925,102,138,184,159,163
4,DRR146894,NODE_4230_length_1601_cov_1210.913325_DRR146894,NODE_4230_length_1601_cov_1210.913325_DRR146894_2,NP_062916.1_coat_protein_[Tobacco_mild_green_m...,0.201389,6,0.905660,115,144,174,159,159
...,...,...,...,...,...,...,...,...,...,...,...,...
154195,SRR9665731,NODE_2_length_3626_cov_0.985424_SRR9665731,NODE_2_length_3626_cov_0.985424_SRR9665731_6,OG_CAA26998.1_unnamed_protein_product_[Pepper_...,0.235294,9,0.850000,78,102,241,120,223
154196,SRR9665731,NODE_2_length_3626_cov_0.985424_SRR9665731,NODE_2_length_3626_cov_0.985424_SRR9665731_6,OG_CAA85421.1_22.3K_[Tobacco_rattle_virus],0.273585,12,0.883333,77,106,218,120,204
154197,SRR9665731,NODE_2_length_3626_cov_0.985424_SRR9665731,NODE_2_length_3626_cov_0.985424_SRR9665731_6,OG_CAA27484.1_unnamed_protein_product_[Barley_...,0.279279,12,0.925000,80,111,207,120,198
154198,SRR9665731,NODE_2_length_3626_cov_0.985424_SRR9665731,NODE_2_length_3626_cov_0.985424_SRR9665731_6,OG_CAA86469.1_beta-A_protein_[Lychnis_ringspot...,0.264957,11,0.975000,86,117,198,120,195


In [6]:
dfs['mp_orf_x_ref']

,SRR,contig_name,orf_name,ref_name,identity_score,gap_openings,N/orf_len,M,N,aln_len,orf_len,ref_len
0,DRR146894,NODE_7350_length_1000_cov_719.098413_DRR146894,NODE_7350_length_1000_cov_719.098413_DRR146894_1,NP_056811.1_34KDa_movement_protein_[Odontoglos...,0.260504,19,0.739130,176,238,387,322,303
1,DRR146894,NODE_7350_length_1000_cov_719.098413_DRR146894,NODE_7350_length_1000_cov_719.098413_DRR146894_1,YP_001333652.1_movement_protein_[Bell_pepper_m...,0.255411,17,0.717391,172,231,369,322,278
2,DRR146894,NODE_7350_length_1000_cov_719.098413_DRR146894,NODE_7350_length_1000_cov_719.098413_DRR146894_1,YP_009553015.2_movement_protein_[Opuntia_virus_2],0.287671,16,0.680124,156,219,357,322,254
3,DRR146894,NODE_7350_length_1000_cov_719.098413_DRR146894,NODE_7350_length_1000_cov_719.098413_DRR146894_1,YP_004956729.1_unnamed_protein_product_[Clitor...,0.240816,18,0.760870,186,245,373,322,296
4,DRR146894,NODE_7350_length_1000_cov_719.098413_DRR146894,NODE_7350_length_1000_cov_719.098413_DRR146894_1,NP_062915.1_28.5_kDa_protein_[Tobacco_mild_gre...,0.272727,16,0.614907,144,198,380,322,256
...,...,...,...,...,...,...,...,...,...,...,...,...
32555,SRR9960038,NODE_10536_length_2129_cov_1.942008_SRR9960038,NODE_10536_length_2129_cov_1.942008_SRR9960038_1,BDB16632.1_movement_protein[Scopolia_mild_mott...,0.248756,16,0.764259,151,201,327,263,265
32556,SRR9960038,NODE_10536_length_2129_cov_1.942008_SRR9960038,NODE_10536_length_2129_cov_1.942008_SRR9960038_1,UZH96201.1_movement_protein_African_eggplant-a...,0.279793,21,0.733840,139,193,328,263,258
32557,SRR9960038,NODE_10536_length_2129_cov_1.942008_SRR9960038,NODE_10536_length_2129_cov_1.942008_SRR9960038_1,WOC94061.1_movement_protein_[Bertoki_tobamovir...,0.297872,14,0.714829,132,188,342,263,267
32558,SRR9960038,NODE_10536_length_2129_cov_1.942008_SRR9960038,NODE_10536_length_2129_cov_1.942008_SRR9960038_1,WOC94058.1_movement_protein_[Gorica_tobamoviru...,0.287179,20,0.741445,139,195,337,263,269
